In [74]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [75]:
df = pd.read_csv('/content/merged_2024.csv')

<ipython-input-75-8afcc8a2d8c7>:1: DtypeWarning: Columns (13,15,20,22,34) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/merged_2024.csv')


In [76]:
df.head()

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,TAIL_NUM,ORIGIN_AIRPORT_ID,...,ARR_DELAY_NEW,ARR_TIME_BLK,CANCELLED,CANCELLATION_CODE,AIR_TIME,FLIGHTS,DISTANCE,WEATHER_DELAY,OP_CARRIER_FL_NUM,CRS_ELAPSED_TIME
0,2024,1,1,1,1,1/1/2024 12:00:00 AM,AA,19805,N102UW,11292,...,0.0,1400-1459,0.0,NaN,94.0,1.0,602.0,NaN,NaN,NaN
1,2024,1,1,1,1,1/1/2024 12:00:00 AM,AA,19805,N102UW,14107,...,0.0,1100-1159,0.0,NaN,82.0,1.0,602.0,NaN,NaN,NaN
2,2024,1,1,1,1,1/1/2024 12:00:00 AM,AA,19805,N141NN,11298,...,12.0,2300-2359,0.0,NaN,90.0,1.0,641.0,NaN,NaN,NaN
3,2024,1,1,1,1,1/1/2024 12:00:00 AM,AA,19805,N144AN,11292,...,0.0,1500-1559,0.0,NaN,86.0,1.0,641.0,NaN,NaN,NaN
4,2024,1,1,1,1,1/1/2024 12:00:00 AM,AA,19805,N144AN,11298,...,17.0,1100-1159,0.0,NaN,97.0,1.0,641.0,0.0,NaN,NaN


In [77]:
df.shape

(107198, 41)

In [78]:
df['ORIGIN_STATE_ABR'].unique()

array(['CO', 'AZ', 'TX', 'NC', 'FL', 'IL', 'NY', 'PA', 'CA', 'WA', 'OR',
       'MA', 'GA', 'UT', 'MN', 'MI', 'VA', 'AR', 'OK', 'KY', 'MO', 'TN',
       'NV', 'WI', 'IN', 'MD', 'NE', 'LA', 'ND', 'SD', 'MT', 'ID', 'AL',
       'IA', 'SC', 'KS', 'WY', 'OH', 'HI', 'NJ', 'NM', 'AK', 'PR', 'CT',
       'ME', 'RI', 'VT', 'VI'], dtype=object)

In [79]:
def convert_hhmm_to_time(hhmm):
    if pd.isnull(hhmm) or hhmm == '':
        return None
    hhmm = int(hhmm)
    hours = hhmm // 100
    minutes = hhmm % 100
    # Handle cases where hhmm is invalid
    if hours >= 24 or minutes >= 60:
        return None
    return datetime.strptime(f'{hours:02d}:{minutes:02d}', '%H:%M').time()


In [80]:
def get_departure_datetime(row):
    # Extract FL_DATE
    FL_DATE = row['FL_DATE']

    flight_date = pd.to_datetime(FL_DATE).date()
    # Extract CRS_DEP_TIME
    DEP_TIME = row.get('CRS_DEP_TIME')

    # Use scheduled time if actual time is missing
    if pd.isnull(DEP_TIME) or DEP_TIME == '':
        DEP_TIME = row.get('DEP_TIME')

    # Convert time to datetime.time object
    dep_time = convert_hhmm_to_time(DEP_TIME)

    # Combine flight_date and dep_time to get departure_datetime
    departure_datetime = datetime.combine(flight_date, dep_time)

    return pd.Series({'departure_datetime': departure_datetime})


In [81]:
# Apply the function to each row in the DataFrame
# df[['departure_datetime']] = df.apply(get_departure_datetime, axis=1)

df['departure_datetime'] = df.apply(lambda row: get_departure_datetime(row), axis=1)

In [82]:
df.to_csv('updated_merged_2024.csv',index=False)

In [83]:
# Load the CSV files
weather_df = pd.read_csv('/content/stations_20240101_20240201_filtered.csv')  # Weather CSV file path
flights_df = pd.read_csv('/content/updated_merged_2024.csv')  # Flights CSV file path

# Drop unwanted columns from the weather data
weather_df.drop(columns=['ice_accretion_1hr', 'ice_accretion_3hr', 'ice_accretion_6hr'], inplace=True)

# Convert the 'valid' column in weather data (24-hour format)
weather_df['valid'] = pd.to_datetime(weather_df['valid'], format='%Y-%m-%d %H:%M')
flights_df['departure_datetime'] = pd.to_datetime(flights_df['departure_datetime'], format='%Y-%m-%d %H:%M:%S')

# Round to the nearest hour (this will automatically handle date increments)
flights_df['rounded_departure'] = flights_df['departure_datetime'].dt.round('H')
weather_df['rounded_valid'] = weather_df['valid'].dt.round('H')

#Dropping duplicates row after rounding up time
weather_df[['station','rounded_valid']].drop_duplicates(inplace = True)

# Perform the left join on the entire flights_df, matching on 'rounded_departure', 'rounded_valid', and 'ORIGIN' with 'station'
merged_df = pd.merge(
    flights_df,
    weather_df,
    left_on=['rounded_departure', 'ORIGIN'],
    right_on=['rounded_valid', 'station'],
    how='left'
)


<ipython-input-83-04499089fa9d>:3: DtypeWarning: Columns (13,15,20,22,34) have mixed types. Specify dtype option on import or set low_memory=False.
  flights_df = pd.read_csv('/content/updated_merged_2024.csv')  # Flights CSV file path
<ipython-input-83-04499089fa9d>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_df[['station','rounded_valid']].drop_duplicates(inplace = True)


In [84]:
merged_df.to_csv('final_merged_2024.csv',index=False)